# Attractor Neural Network  
## Amari-Hopfield Network  
Let's begin from studying the one of the most classic attractor neural network model -- **Amari-Hopfield Network**.  

This network consists of $N$ neurons, each state is denoted as **$S_i (i=1,2,...,N)$**. This model assumes the neuron can only be at **two binary states, i.e., $S_i = +1 / -1$**, representing the neuron can only be either activated or silenced at one time. The neurons are recurrently connected with each other, the connection strength can be expressed as a matrix **$W$**.  
The update rule of the network is expressed as,  
$$S_i(t+1) = sign(\sum_{j=1}^N W_{ij}S_j(t) - \theta)$$  
where $\sum_{j=1}^N W_{ij}S_j(t)$ represents the total recurrent input current the $i_{th}$ neuron receives at time $t$, which in turn determines the neuron state in the next time. Specifically, if the recurrent input exceeds a certain threshold $\theta$, the neuron state $S_i=1$, otherwise $S_i = -1$.  
It should be noted that, the update of different neurons are asynchoronous, which means that at each time only one neuron state will change.  

Amari and Hopfield proved that, to store $n$ different **patterns $\xi^\mu (\mu = 1,2,...n)$** as attractor states in this network, the synaptic weights between i-th an j-th neuron $W_{ij}$ should be:  
$$W_{ij} = \sum_{\mu=1}^n \xi_i^\mu \xi_j^\mu$$  
where $W_{ii}=0$, representing that neurons do not connect with itself.  

The energy function $E$ is expressed as,  
$$ E = -\frac{1}{2} \sum_{i,j}W_{ij} S_i S_j + \theta \sum_i S_i

## Costomize Amari-Hopfield Network

In [5]:
import numpy as np
import matplotlib.pyplot as plt

import brainpy as bp
import brainpy.math as bm

class HopfieldNet(bp.DynamicalSystem):
  def __init__(self, num):
    super().__init__()
    self.num = num
    self.weight = bm.Variable(bm.zeros([num, num]))

  # Train function for the Hopfield net. (By updating the weights)
  @bm.cls_jit
  def store_patterns(self, samples):
    # data: An array with [d, N]. 'd' is the number of data samples used to train. (In this case 2)
    assert samples.ndim == 2
    assert samples.shape[1] == self.num

    # Loop through all data samples.
    bm.for_loop(self.store, samples)

    # Hopfield nets are a form of RNNs, albeit without self-connections, and so,
    # we need to make sure that the diagonal elements of the final weight matrix are zero.
    bm.fill_diagonal(self.weight, 0)

  # Storing one sample pattern
  @bm.cls_jit
  def store(self, sample):
    # sample is an array with the shape of (N,)
    assert self.num == sample.shape[0]

    # Data cross-product gives neural hopfield update rule.
    w_update = bm.outer(sample, sample)

    # Sum all pattern cross-products.
    self.weight += w_update

  def async_recover(self, sample, n, energy=False):
    # n: the number of iterations to recover
    # energy: calculate the energy function
    idxs = bm.random.randint(0, self.num, n)  # the sampled positions
    # JIT compilation requires to label the value to be changed as Variable
    sample = bm.Variable(sample)

    def recover(i):
      # i: the position to update
      # update
      sample[i] = bm.sign(bm.inner(self.weight[i], sample))
      # return energy
      if energy:
        energy_trace = self.energy(sample)
        return energy_trace
    r = bm.for_loop(recover, idxs, progress_bar=True) # for loop JIT
    return (sample, r) if energy else sample

    r = bm.for_loop(recover, idxs)  # for loop JIT
    return (sample, r) if energy else sample

  # This function computes the Hopfield nets' energy.
  def energy(self, x):
    # x: [N] data vector
    return bm.inner(- x @ self.weight, x)


## Simulating the Amari-Hopfield Network  
Now let us test its ability to store patterns and steadily retrieve it as attractor states!  
### Data for testing  
Now we are going to load the data in. We have already saved off two images, a ‘9’ and a ‘4’, to keep things simple for us. We also reshape them into their native [28 x 28] size, so that we can later corrupt them in this domain, and then reconstruct them.  

The data can be downloaded in this url: https://github.com/brainpy/examples/blob/main/attractors/data/data_to_train_on.npy

In [6]:
data = (np.load('/home/mw/project/data_to_train_on.npy')).astype(np.float32)

data.shape

(2, 784)

What does our data look like? Well they are just two binary images, showing a figure ‘9’, and a figure ‘4’. Let’s image them just to take a peek at what they look like.

In [7]:
# Show the training data - it's just two different samples that we are going to want the Hopfield net to store.
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15,7.5))
ax[0].imshow(data[0].reshape(28, 28), interpolation='None', cmap='winter')
ax[1].imshow(data[1].reshape(28, 28), interpolation='None', cmap='winter')
plt.show()

<Figure size 1080x540 with 2 Axes>

We will use this function to take in a good image, and corrupt it, based on two types of noise, either bernouli noise, (where every pixel is randomly flipped in sign), or masking noise, where we blot out entire sections of the image. Our Hopfield net will be able to recover the original image via energy minimization as we will see.

In [8]:
def add_bernouli_noise(img, prob=0.1, mask_value=1.):
    rand = np.random.rand(*img.shape) < prob
    return np.where(rand, mask_value, img)
def add_mask_noise(img, mask_value=-1, mask_matrix=np.zeros((2,2))):
    img = np.copy(img)
    img[mask_matrix[0,0]: mask_matrix[0,1], mask_matrix[1,0]: mask_matrix[1,1]] = mask_value
    return img

### Training  
OK this is where the fun begins! We are now going to train our Hopfield net. The objective is simple: We want it to store those two patterns we are feeding it, such that a faithful reconstruction can be performed, when corrupted versions of either of those inputs is given later in the future.

In [9]:
# Train the Hopfield net!

# So there are as many neurons as pixels per pattern, and this is what this computes.
net = HopfieldNet(num=data.shape[1])

# Great - we are all set - now we train!
net.store_patterns(data)

Now at this point, we have a trained weight matrix, encoded with the pattern ‘4’ and pattern ‘9’. (We will see later on that Hopfield nets aren’t that memory efficient, but it’s still amazing what they are capable of). Now remember - the objective here is simple: We want it to store those two patterns we are feeding it, such that faithful reconstruction can later be performed, when corrupted versions of either of those inputs are presented. But does it work? Let’s find out!).  

### Testing  
We can corrupt our data in many ways - and here, we can add two types of noise: Bernouli noise, (where every pixel is flipped in sign with a certain probability) and/or masking noise, (where we simply blot out certain regions with one particular sign).

In [10]:
# Let's pick one the image '4', and corrupt it.
image = data[1]

# Corrupt the image with bernouli noise.
corrupted_image = add_bernouli_noise(image, prob=0.1)

# Let's see the original image, as well as the corrupted version.
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15,7.5))
ax[0].imshow(image.reshape(28, 28), interpolation='None', cmap='winter')
ax[1].imshow(corrupted_image.reshape(28, 28), interpolation='None', cmap='winter')
plt.show()

<Figure size 1080x540 with 2 Axes>

This is the moment of truth: With a corrupted input now at hand, we then present this to our trained net, and hope that it denoises it. We do so by picking pixels at random, and then evaluating their activation functions, based on the current input configuration, (corrupted image), and the set weights, (that do not change here, as the net has already been trained).  

Since the Hopfield net stores patterns as minimal enery states, every iteration where we update a neuron/pixel should place the net into a lower energy configuration - that is - closer to the original pattern we stored - the weights are designed to do so after all.

In [11]:

# The number of iterations where we randomly update neurons/pixels
n_iterations = 5000

# Loop through, and recover the image from it's corrupted self.
# energy_vec: This will store the energy of the Hopfield net.
cleaned_image, energy_vec = net.async_recover(corrupted_image, n_iterations, True)

# (For imaging purposes)
fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize=(15, 7.5))
ax[0].imshow(corrupted_image.reshape(28, 28), interpolation='None', cmap='winter')
ax[0].set_title('Corrupted Image')
ax[1].imshow(cleaned_image.reshape(28, 28), interpolation='None', cmap='winter')
ax[1].set_title('Recovered Image')
plt.show()

  0%|          | 0/5000 [00:00<?, ?it/s]

<Figure size 1080x540 with 2 Axes>

The Hopfield energy plotted as the recovery iteration index is shown below. We can see that as the recovery process was being run, the total net energy was being decreased.

In [12]:
# Plot the Hopfield energy during recovery
plt.plot(bm.as_numpy(energy_vec))
plt.grid('on')
plt.show()

<Figure size 432x288 with 1 Axes>

Our original pattern has been very nicely recovered. What about masking noise and bernouli noise? 

In [13]:
# Masking and Bernouli noise
mask_matrix = np.asarray([[7, 20], [8, 25]], dtype=np.int32)

image = data[0].reshape(28, 28)
corrupted_image = add_mask_noise(image, mask_value=1, mask_matrix=mask_matrix)
corrupted_image = add_bernouli_noise(corrupted_image, prob=0.1)

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15,7.5))
ax[0].imshow(image, interpolation='None', cmap='winter')
ax[1].imshow(corrupted_image, interpolation='None', cmap='winter')
plt.show()

<Figure size 1080x540 with 2 Axes>

The rest part is leaved to you as homework.  
## Try it by your own hand and hope you enjoy it!

# Modern Hopfield Network  
The traditional Hopfield network has a lot of limits as a memory storage system, including its limited capacity and the lack of ability for representing continuous variables.  

To overcome these limits, "modern Hopfield Network" is proposed. In contrast with the traditional Hopfield Network, Modern Hopfield is two-layerd network.  

Notations: external input of visible neurons, $I_i(t)$; synaptic input of visible neurons, $v_i(t)$; firing rate of visible neurons, $g(v_i(t))$; synaptic input of hidden neurons, $h_{\mu}(t)$; firing rate of visible neurons, $f(h_{\mu}(t))$;  

The dynamics of the network is given by:  

$$ \tau_f \frac{dv_i(t)}{dt} = -v_i(t)+\sum_{\mu=1}^{N_h} \xi_{i, \mu}f(h_{\mu}(t))+I_i(t)$$  
$$ \tau_h \frac{dh_{\mu}(t)}{dt} = -h_{\mu}(t)+\sum_{i=1}^{N_f} \xi_{\mu, i}g(v_{i}(t))$$  

The general form of energy fucntion is given by:  

$$ E(t) =  \left[\sum_{i=1}^{N_f}(x_i(t)-I_i(t))g_i(x_i(t)) - L_v(f_i(t))\right] + \left[\sum_{\mu=1}^{N_h}h_\mu(t) f_\mu(t) - L_h(h_\mu(t))\right] - \sum_{\mu, i} f(h_{\mu}(t))\xi_{\mu, i}g(v_{i}(t))$$  
where $L_h$ and $L_v$ represents two Larangian functions of the activation function, which is given by:  
$$f_\mu = \frac{\partial L_\mu}{\partial h_\mu},        g_i = \frac{\partial L_v}{\partial v_i} $$  

# Costomize a Modern Hopfield Network  
Here we assume that the Lagrangian fucntions take the form,  
$$L_\mu = log\left(\sum_{\mu} e^{h_\mu} \right),        L_v = \frac{1}{2}\sum_{i}v_i^2 $$  
which gives,  
$$f_\mu(h) = softmax(h_\mu) = \frac{e^{h_\mu}}{\sum_{\mu}e^{h_\mu}}  ,        g_i(v) = v_i $$

## Costomize a Modern Hopfield Network

In [14]:

class ModernHopfieldNet(bp.DynamicalSystem):
  def __init__(self, Patterns, tau_f = 100., tau_h = 1.):
    super().__init__()
    self.num_f = Patterns.shape[1]
    self.num_h = Patterns.shape[0]
    self.tau_f = tau_f
    self.tau_h = tau_h
    self.conn_mat = bm.Variable(Patterns)
    self.v = bm.Variable(self.num_f)
    self.g = bm.Variable(self.num_f)
    self.h = bm.Variable(self.num_h)
    self.f = bm.Variable(self.num_h)
    self.E = bm.Variable(1)

    self.integral = bp.odeint(self.derivative)

  # 微分方程
  @property
  def derivative(self):
      dv = lambda v, t, I_ext: (-v + bm.matmul(bm.transpose(self.conn_mat), self.f) + I_ext) / self.tau_f
      dh = lambda h, t: (-h + bm.matmul(self.conn_mat,self.g) ) / self.tau_h
      return bp.JointEq([dv, dh])
  
  def softmax(self, x1):
    x1 = x1 - bm.max(x1)
    # x1 = bm.where(x1<=50, x1, 50)
    return bm.exp(x1)/bm.sum(bm.exp(x1))

  def activation(self):
    h = self.h.value
    self.f.value = self.softmax(h)
    self.g = self.v
  
  def initialize(self, sample):
    self.v = bm.Variable(sample)
    self.g = self.v
    h = bm.matmul(self.conn_mat, self.g)
    self.h.value = h
    self.f = self.softmax(self.h)
  
  # This function computes the Hopfield nets' energy.
  def energy(self):
    h = self.h.value
    hmax = bm.max(h)
    exp_h = bm.exp(self.h-hmax)
    v2 = bm.square(self.v)
    E = bm.array(1/2*bm.sum(v2) - hmax - bm.log(bm.sum(exp_h)))
    self.E.value = E.reshape(-1)

  def update(self, input):
    self.activation()
    v, h = self.integral(self.v, self.h, bp.share['t'], input)
    self.v = v
    self.h = h
    self.energy()

### Import few pictures that we need to store and add noise

In [15]:

from PIL import Image
# 打开原始图像
n_image = 4
n1 = 192*2
n2 = 108*2
data = np.zeros([n_image, n1 * n2])
image = np.zeros([n_image, n1, n2])
corrupted_image = np.zeros([n_image, n1, n2])
for i in range(n_image):
    original_image_path = '/home/mw/project/bro_' + str(i+1) +  '.jpg'
    original_image = Image.open(original_image_path)
    # Downsampling
    resized_image = original_image.resize((n2, n1), Image.LANCZOS)
    # 将降采样后的图像转换为NumPy数组
    array_image = np.array(resized_image)
    resized_image_array = array_image/256
    image[i] = np.mean(resized_image_array, axis=2)
    data[i] = image[i].reshape([1,-1])
    corrupted_image[i] = add_bernouli_noise(image[i], prob=0.4)

retrieve_idx = 1
# Let's see the original image, as well as the corrupted version.
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15,7.5))
ax[0].imshow(image[retrieve_idx], interpolation='None', cmap='winter', vmin = np.min(image), vmax = np.max(image))
ax[1].imshow(corrupted_image[retrieve_idx], interpolation='None', cmap='winter', vmin = np.min(image), vmax = np.max(image))


<Figure size 1080x540 with 2 Axes>

In [16]:

# run net
net = ModernHopfieldNet(data)
sample = corrupted_image[retrieve_idx].reshape(-1,)
net.initialize(sample=sample)

def run_net(i):  # 20 x size
    net.step_run(i, input=0)
    return net.v, net.E
total_time = 3000
indices = np.arange(total_time)
V_trace, E_trace = bm.for_loop(run_net, indices, progress_bar=True)
# First set up the figure, the axis, and the plot element we want to animate

fig = plt.figure()
plt.plot(E_trace)

from matplotlib.animation import FuncAnimation
fps = 30
n_step = 20
# 动画化网络活动\
data1 = bm.as_numpy(V_trace[::n_step, :])
T = data1.shape[0]
snapshots = bm.as_numpy(data1).reshape([T, n1, n2])


# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots(1,3,figsize=(16,8))
ax[1].imshow(image[retrieve_idx], interpolation='None', cmap='winter', vmin = np.min(image), vmax = np.max(image))
ax[1].set_title('clean image')
ax[2].imshow(corrupted_image[retrieve_idx], interpolation='None', cmap='winter', vmin = np.min(image), vmax = np.max(image))
ax[2].set_title('noise image')
a = snapshots[0]
im = ax[0].imshow(a, interpolation='none', cmap='winter', vmin = np.min(image), vmax = np.max(image))
ax[0].set_title('network response')
def animate_func(i):
    im.set_array(snapshots[i])
    return [im]

anim = FuncAnimation(fig, animate_func, frames = T, interval = 1000 / fps, blit=True)
plt.show()

  0%|          | 0/3000 [00:00<?, ?it/s]

<Figure size 432x288 with 1 Axes>

<Figure size 1152x576 with 3 Axes>

# Continuous Attractor Neural Network  
## The dynamics of the model  
### Synaptic current dynamics (exponential conductance model):  
$$  
\tau\frac{dU(x,t)}{dt} = -U(x,t)+I^{rec} (x,t)+ I^{ext}(x,t)  
$$  
where the recurrent input $I_{rec}(x,t) = \int_{x'}J(x,x')r(x',t)dx'$, which represents a weighted sum of all recurrently connected neurons' firing rate,  
### Activation function (which transfer the synaptic current to firing rates):  
$$  
r(x,t) = \frac{U(x,t)^2}{1+k\int_x' U(x',t)^2dx'}  
$$  
### Synaptic connection strength matrix  
$$  
J(x,t) = J_0 exp [-\frac{dist(x，x')^2}{2a^2}]  
$$  
where $dist(x, x')$ represents the distance of two features, e.g., in Euler space the $dist(x,x') = |x-x'|$.  
## Customize a ring CANN in brainpy  
In simulations, we can not simulate a CANN encoding features ranging $(-\inf, \inf)$. Instead, we simulate a ring attractor network which encodes features ranging $(-\pi, \pi)$. Note that the distance on a ring should be:  
$$  
dist_{ring}(x,x') = min(|x-x'|,2\pi-|x-x'|)  
$$  


In [17]:
import brainpy as bp
import brainpy.math as bm
import matplotlib.pyplot as plt

plt.rcParams.update({"font.size": 15})
plt.rcParams['font.sans-serif'] = ['Times New Roman']


class CANN1D(bp.NeuGroupNS):
  def __init__(self, num, tau=1., k=8.1, a=0.5, A=10., J0=4.,
               z_min=-bm.pi, z_max=bm.pi, **kwargs):
    super(CANN1D, self).__init__(size=num, **kwargs)

    # 初始化参数
    self.tau = tau
    self.k = k
    self.a = a
    self.A = A
    self.J0 = J0

    # 初始化特征空间相关参数
    self.z_min = z_min
    self.z_max = z_max
    self.z_range = z_max - z_min
    self.x = bm.linspace(z_min, z_max, num)
    self.rho = num / self.z_range
    self.dx = self.z_range / num

    # 初始化变量
    self.u = bm.Variable(bm.zeros(num))
    self.input = bm.Variable(bm.zeros(num))
    self.conn_mat = self.make_conn(self.x)  # 连接矩阵

    # 定义积分函数
    self.integral = bp.odeint(self.derivative)

  # 微分方程
  @property
  def derivative(self):
    du = lambda u, t, Irec, Iext: (-u + Irec + Iext) / self.tau
    return du

  # 将距离转换到[-z_range/2, z_range/2)之间
  def dist(self, d):
    d = bm.remainder(d, self.z_range)
    d = bm.where(d > 0.5 * self.z_range, d - self.z_range, d)
    return d

  # 计算连接矩阵
  def make_conn(self, x):
    assert bm.ndim(x) == 1
    d = self.dist(x - x[:, None])  # 距离矩阵
    Jxx = self.J0 * bm.exp(
      -0.5 * bm.square(d / self.a)) / (bm.sqrt(2 * bm.pi) * self.a) 
    return Jxx

  # 获取各个神经元到pos处神经元的输入
  def get_stimulus_by_pos(self, pos):
    return self.A * bm.exp(-0.25 * bm.square(self.dist(self.x - pos) / self.a))

  def update(self, x=None):
    _t = bp.share['t']
    u2 = bm.square(self.u)
    r = u2 / (1.0 + self.k * bm.sum(u2))
    Irec = bm.dot(self.conn_mat, r)
    self.u[:] = self.integral(self.u, _t,Irec, self.input)
    self.input[:] = 0.  # 重置外部电流

## Simulate the persistent activity of CANN after the removal of external input  
We first project a gaussian-bump external input to activate CANN, then remove it and check the population response of CANN.

In [18]:
def Persistent_Activity(k=0.1,J0=1.):
    # 生成CANN
    cann = CANN1D(num=512, k=k,J0=J0)

    # 生成外部刺激，从第2到12ms，持续10ms
    dur1, dur2, dur3 = 2., 10., 10.
    I1 = cann.get_stimulus_by_pos(0.)
    Iext, duration = bp.inputs.section_input(values=[0., I1, 0.],
                                             durations=[dur1, dur2, dur3],
                                             return_length=True)
    noise_level = 0.1
    noise = bm.random.normal(0., noise_level, (int(duration / bm.get_dt()), len(I1)))
    Iext += noise
    # 运行数值模拟
    runner = bp.DSRunner(cann, inputs=['input', Iext, 'iter'], monitors=['u'])
    runner.run(duration)

    # 可视化
    def plot_response(t):
        fig, gs = bp.visualize.get_figure(1, 1, 4.5, 6)
        ax = fig.add_subplot(gs[0, 0])
        ts = int(t / bm.get_dt())
        I, u = Iext[ts], runner.mon.u[ts]
        ax.plot(cann.x, I, label='Iext')
        ax.plot(cann.x, u, linestyle='dashed', label='U')
        ax.set_title(r'$t$' + ' = {} ms'.format(t))
        ax.set_xlabel(r'$x$')
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.legend()
        # plt.savefig(f'CANN_t={t}.pdf', transparent=True, dpi=500)

    plot_response(t=10.)
    plot_response(t=20.)

    bp.visualize.animate_1D(
        dynamical_vars=[{'ys': runner.mon.u, 'xs': cann.x, 'legend': 'u'},
                        {'ys': Iext, 'xs': cann.x, 'legend': 'Iext'}],
        frame_step=1,
        frame_delay=40,
        show=True,
    )
    plt.show()
Persistent_Activity(k=0.1)

  0%|          | 0/220 [00:00<?, ?it/s]

findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
fin

<Figure size 432x324 with 1 Axes>

findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following fa

<Figure size 432x324 with 1 Axes>

findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following fa

<Figure size 432x432 with 1 Axes>

## Simulate the tracking behavior of CANN  
We first project a moving gaussian-bump input to CANN and check the population response of CANN.

In [19]:

def smooth_tracking():
    cann = CANN1D(num=512, k=8.1)

    # 定义随时间变化的外部刺激
    v_ext = 1e-3
    dur1, dur2, dur3 = 10., 10., 20
    num1 = int(dur1 / bm.get_dt())
    num2 = int(dur2 / bm.get_dt())
    num3 = int(dur3 / bm.get_dt())
    position = bm.zeros(num1 + num2 + num3)
    position[num1: num1 + num2] = bm.linspace(0., 1.5 * bm.pi, num2)
    position[num1 + num2: ] = 1.5 * bm.pi
    position = position.reshape((-1, 1))
    Iext = cann.get_stimulus_by_pos(position)

    # 运行模拟
    runner = bp.DSRunner(cann,
                         inputs=['input', Iext, 'iter'],
                         monitors=['u'])
    runner.run(dur1 + dur2 + dur3)

    # 可视化
    def plot_response(t, extra_fun=None):
        fig, gs = bp.visualize.get_figure(1, 1, 4.5, 6)
        ax = fig.add_subplot(gs[0, 0])
        ts = int(t / bm.get_dt())
        I, u = Iext[ts], runner.mon.u[ts]
        ax.plot(cann.x, I, label='Iext')
        ax.plot(cann.x, u, linestyle='dashed', label='U')
        ax.set_title(r'$t$' + ' = {} ms'.format(t))
        ax.set_xlabel(r'$x$')
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.legend()
        if extra_fun: extra_fun()
        # plt.savefig(f'CANN_tracking_t={t}.pdf', transparent=True, dpi=500)

    plot_response(t=10.)

    def f():
        plt.annotate('', xy=(1.5, 10), xytext=(0.5, 10), arrowprops=dict(arrowstyle="->"))

    plot_response(t=15., extra_fun=f)

    def f():
        plt.annotate('', xy=(-2, 10), xytext=(-3, 10), arrowprops=dict(arrowstyle="->"))

    plot_response(t=20., extra_fun=f)
    plot_response(t=30.)

    bp.visualize.animate_1D(
        dynamical_vars=[{'ys': runner.mon.u, 'xs': cann.x, 'legend': 'u'},
                        {'ys': Iext, 'xs': cann.x, 'legend': 'Iext'}],
        frame_step=5,
        frame_delay=50,
        show=True,
    )
    plt.show()
smooth_tracking()

  0%|          | 0/400 [00:00<?, ?it/s]

/opt/conda/lib/python3.9/site-packages/matplotlib/animation.py:884: UserWarning: Animation was deleted without rendering anything. This is most likely not intended. To prevent deletion, assign the Animation to a variable, e.g. `anim`, that exists until you output the Animation using `plt.show()` or `anim.save()`.
  warnings.warn(
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were f

<Figure size 432x324 with 1 Axes>

findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following fa

<Figure size 432x324 with 1 Axes>

...

<Figure size 432x324 with 1 Axes>

findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following fa

<Figure size 432x324 with 1 Axes>

findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following families were found: Times New Roman
findfont: Generic family 'sans-serif' not found because none of the following fa

<Figure size 432x432 with 1 Axes>